In [2]:
from pandas.io import gbq
import pandas as pd
import matplotlib.pyplot as plt

bq_project = "thesis-jaeyoon"

In [26]:
gp_query="""

select
	p.publication_number,
	p.application_number,
	p.country_code,
  p.publication_date,
  CAST(FLOOR(p.filing_date / 10000) AS INT64) AS filing_year,
  CAST(FLOOR(p.priority_date / 10000) AS INT64) AS priority_year,
  p.family_id,
  p.entity_status,
	STRING_AGG(DISTINCT ipcs.code) as ipc_codes,
  STRING_AGG(DISTINCT uspcs.code) as uspc_codes,
  STRING_AGG(DISTINCT pc.application_number) as priority_application_number,
  STRING_AGG(DISTINCT cit.publication_number) as citations,
  STRING_AGG(DISTINCT ah.name, "||") as assignees_harmonized,
  STRING_AGG(DISTINCT title.text, "||") as title_text,
  STRING_AGG(DISTINCT abstract.text, "||") as abstract_text
FROM
  `patents-public-data.patents.publications` AS p
LEFT JOIN UNNEST(p.ipc) AS ipcs
LEFT JOIN UNNESt(p.uspc) AS uspcs
LEFT JOIN UNNEST(p.assignee) AS assignees
LEFT JOIN UNNEST(p.title_localized) AS title
LEFT JOIN UNNEST(p.abstract_localized) AS abstract
LEFT JOIN UNNEST(p.priority_claim) AS pc 
LEFT JOIN UNNEST(p.citation) as cit
LEFT JOIN UNNEST(p.assignee_harmonized) as ah
LEFT JOIN UNNEST(p.description_localized) as description
where 
  pc.application_number = p.application_number AND 
  (
  ipcs.code LIKE '%A61J1/%' OR ipcs.code LIKE '%A61J3/%' OR
  ipcs.code LIKE '%A61K9/%' OR ipcs.code LIKE '%A61K31/%' OR
  ipcs.code LIKE '%A61K33/%' OR ipcs.code LIKE '%A61K35/%' OR
  ipcs.code LIKE '%A61K36/%' OR ipcs.code LIKE '%A61K41/%' OR
  ipcs.code LIKE '%A61K49/%' OR ipcs.code LIKE '%A61K51/%' OR
  ipcs.code LIKE '%A61K39/%' OR ipcs.code LIKE '%A61K45/%' OR
  ipcs.code LIKE '%A61K47/%' OR ipcs.code LIKE '%A61K48/%' OR
  ipcs.code LIKE '%A61P/%' OR
  uspcs.code LIKE '%424/%' OR uspcs.code LIKE '%514/%' 
  )
GROUP BY 
  p.publication_number, p.application_number, p.country_code, p.publication_date, p.filing_date,
	p.priority_date, p.family_id, p.entity_status
"""

In [27]:
gp_df = gbq.read_gbq(
            query=gp_query,
            project_id=bq_project,
            verbose=True,
            dialect='standard')

/Users/jaeyoonjung/anaconda3/lib/python3.6/site-packages/pandas/io/gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


In [28]:
gp_df.to_csv("data/priority_claims.csv", sep="\t", index=False)

In [30]:
ob_patent_dir = "orangebook_chembl.csv"
ob_patent = pd.read_csv(ob_patent_dir, sep="\t")

In [31]:
where_clause = ",".join("'" + ob_patent.publication_number.unique() + "'")

In [32]:
ob_query = '''
select
	p.publication_number,
	p.application_number,
	p.country_code,
  p.publication_date,
  CAST(FLOOR(p.filing_date / 10000) AS INT64) AS filing_year,
  CAST(FLOOR(p.priority_date / 10000) AS INT64) AS priority_year,
  p.family_id,
  p.entity_status,
	STRING_AGG(DISTINCT ipcs.code) as ipc_codes,
  STRING_AGG(DISTINCT uspcs.code) as uspc_codes,
  STRING_AGG(DISTINCT pc.application_number) as priority_application_number,
  STRING_AGG(DISTINCT cit.publication_number) as citations,
  STRING_AGG(DISTINCT ah.name, "||") as assignees_harmonized,
  STRING_AGG(DISTINCT title.text, "||") as title_text,
  STRING_AGG(DISTINCT abstract.text, "||") as abstract_text
FROM
  `patents-public-data.patents.publications` AS p
LEFT JOIN UNNEST(p.ipc) AS ipcs
LEFT JOIN UNNESt(p.uspc) AS uspcs
LEFT JOIN UNNEST(p.assignee) AS assignees
LEFT JOIN UNNEST(p.title_localized) AS title
LEFT JOIN UNNEST(p.abstract_localized) AS abstract
LEFT JOIN UNNEST(p.priority_claim) AS pc 
LEFT JOIN UNNEST(p.citation) as cit
LEFT JOIN UNNEST(p.assignee_harmonized) as ah
LEFT JOIN UNNEST(p.description_localized) as description
WHERE
    p.publication_number IN
    (
    {}
    )
GROUP BY 
p.publication_number, p.application_number, p.country_code, p.publication_date, p.filing_date,
	p.priority_date, p.family_id, p.entity_status
'''.format(where_clause)

In [33]:
ob_df = gbq.read_gbq(
            query=ob_query,
            project_id=bq_project,
            verbose=True,
            dialect='standard')

/Users/jaeyoonjung/anaconda3/lib/python3.6/site-packages/pandas/io/gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


In [34]:
ob_df = pd.merge(left = ob_df, right = ob_patent[["publication_number", "mesh_heading"]], on ="publication_number")

In [35]:
ob_df.to_csv("data/ob_patents.csv", sep='\t', index=False)

In [27]:
ob_df = pd.read_csv("data/ob_patents.csv", sep='\t')

In [28]:
ob_df.publication_number

0        US-5424295-A
1       US-7101993-B1
2       US-6407079-B1
3        US-5604229-A
4        US-6162802-A
5        US-5955109-A
6        US-5420319-A
7        US-5223510-A
8       US-6268489-B1
9        US-5922682-A
10       US-5905082-A
11       US-5364842-A
12       US-RE42152-E
13       US-5344932-A
14       US-5387603-A
15       US-6127353-A
16       US-5424286-A
17       US-5532372-A
18       US-5856336-A
19       US-5665772-A
20       US-5681814-A
21       US-5288726-A
22       US-6090799-A
23       US-5997903-A
24       US-5972990-A
25      US-6545040-B1
26       US-5661136-A
27      US-6469012-B1
28      US-6288095-B1
29       US-5362718-A
            ...      
3227    US-9402805-B1
3228    US-9533955-B2
3229    US-9517236-B2
3230    US-9566290-B2
3231    US-9526726-B2
3232    US-9566289-B2
3233    US-9539335-B2
3234    US-9555079-B1
3235    US-9433630-B1
3236    US-9439921-B2
3237    US-9539310-B1
3238    US-9439911-B2
3239    US-9561177-B2
3240    US-9545399-B2
3241    US

## AUTOMAED PATENT LANDSCAPING!!!

In [30]:
matched_patents= pd.read_csv("data/disease_patent_match_og.csv", sep='\t')

In [31]:
matched_patents_fam=pd.read_csv("data/disease_patent_match_extended_orig.csv", sep='\t')

In [33]:
seed = matched_patents_fam.loc[matched_patents_fam.publication_number.isin(matched_patents.publication_number.values)]
seed = seed.loc[seed.country_code=="US"]

In [66]:
seed = matched_patents_fam.loc[matched_patents_fam.publication_number.isin(matched_patents.publication_number.values)]
seed = seed.loc[seed.country_code=="US"]
seed_pubs = []
for s in seed.publication_number.values:
    if  isinstance(s, float):
        continue

    fi = s.index("-")
    li = s[fi+1::].index("-")
    pub = s[fi+1:fi+li+1]
    if pub.isdigit(): #& (len(pub) == 7):
        seed_pubs.append(pub)


In [67]:
len(seed_pubs)

42161

In [68]:
# write training data for APL
import random
random.seed(a=0)
random_pubs = random.sample(seed_pubs, k= 10000)

with open("data/pharma_seed.csv", 'w') as f:
    for p in random_pubs:
        f.write("%s\n" % p)

In [12]:
apl_patent = pd.DataFrame(columns=["publication_number", "score", "abstract_text"])
for i in range(0,3):
    fname="/Users/jaeyoonjung/patents-public-data/models/landscaping/data/pharma/pharma10000/inference{}.csv".format(str(i))
    apl_patent = apl_patent.append(pd.read_csv(fname, sep="\t"), ignore_index=True)
    
    

In [13]:
apl_patent.shape

(75837, 3)

In [6]:
apl_patent = apl_patent.loc[apl_patent.score > 0.5]

In [7]:
apl_patent.publication_number.nunique()

17899

In [8]:
apl_where_clause = ",".join("'" + apl_patent.publication_number.unique() + "'")

In [9]:
apl_query = '''
select
	p.publication_number,
	p.application_number,
	p.country_code,
  p.publication_date,
  CAST(FLOOR(p.filing_date / 10000) AS INT64) AS filing_year,
  CAST(FLOOR(p.priority_date / 10000) AS INT64) AS priority_year,
  p.family_id,
  p.entity_status,
	STRING_AGG(DISTINCT ipcs.code) as ipc_codes,
  STRING_AGG(DISTINCT uspcs.code) as uspc_codes,
  STRING_AGG(DISTINCT pc.application_number) as priority_application_number,
  STRING_AGG(DISTINCT cit.publication_number) as citations,
  STRING_AGG(DISTINCT ah.name, "||") as assignees_harmonized,
  STRING_AGG(DISTINCT title.text, "||") as title_text,
  STRING_AGG(DISTINCT abstract.text, "||") as abstract_text
FROM
  `patents-public-data.patents.publications` AS p
LEFT JOIN UNNEST(p.ipc) AS ipcs
LEFT JOIN UNNESt(p.uspc) AS uspcs
LEFT JOIN UNNEST(p.assignee) AS assignees
LEFT JOIN UNNEST(p.title_localized) AS title
LEFT JOIN UNNEST(p.abstract_localized) AS abstract
LEFT JOIN UNNEST(p.priority_claim) AS pc 
LEFT JOIN UNNEST(p.citation) as cit
LEFT JOIN UNNEST(p.assignee_harmonized) as ah
LEFT JOIN UNNEST(p.description_localized) as description
WHERE
    p.publication_number IN
    (
    {}
    )
GROUP BY 
p.publication_number, p.application_number, p.country_code, p.publication_date, p.filing_date,
	p.priority_date, p.family_id, p.entity_status
'''.format(apl_where_clause)

In [10]:
apl_df = gbq.read_gbq(
            query=apl_query,
            project_id=bq_project,
            dialect='standard')

/Users/jaeyoonjung/anaconda3/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [21]:
apl_df.loc[apl_df.application_number == apl_df.priority_application_number]

,publication_number,application_number,country_code,publication_date,filing_year,priority_year,family_id,entity_status,ipc_codes,uspc_codes,priority_application_number,citations,assignees_harmonized,title_text,abstract_text
0,US-4402430-A,US-19233980-A,US,19830906,1980,1980,22709232,,F41H9/10,"222/183.,222/509.,222/325.,42/1.08",US-19233980-A,"US-3247655-A,DE-1915045-A1,US-3028055-A,US-422...",WASHBURN ALFRED F,Mechanism for dispensing chemicals from a pres...,A hand-held holder for pressurized chemical ca...
3,US-4148308-A,US-80158077-A,US,19790410,1977,1977,25181510,,"A61M16/04,A61B1/24","128/207.14,600/205.,600/239.",US-80158077-A,"US-3683931-A,DE-669841-C,US-3154069-A,DE-57099...",SAYER WILLIAM J,Mouthpiece with a tongue retractor,A mouthpiece comprising a tubular member about...
4,US-4195079-A,US-790179-A,US,19800325,1979,1979,21728712,,"A23K20/195,C12P1/06,C12P17/18","435/825.,424/122.,435/132.",US-790179-A,US-4132778-A,PFIZER,New polycyclic ether antibiotic,A new polycyclic ether antibiotic produced by ...
5,US-4758158-A,US-54423183-A,US,19880719,1983,1983,24171318,small,A61C9/00,433/90.,US-54423183-A,"US-2903794-A,US-4260597-A,US-3345497-A,US-3731...",SOL BELPORT COMPANY INC,Hydrocolloid injection system,The hydrocolloid injection system which utiliz...
6,US-4356822-A,US-19805680-A,US,19821102,1980,1980,22731820,,"A61M5/32,A61M5/46","604/192.,604/198.,604/117.",US-19805680-A,"US-1436707-A,US-3739780-A,US-2568346-A,US-3527...",WINSTEAD HALL DEBORAH,Syringe assembly,A syringe assembly having a hollow barrel and ...
8,US-4380590-A,US-94388978-A,US,19830419,1978,1978,25480433,,"B01J39/04,B01J47/00,C08F8/00,C13B20/14,B01J41/...","977/773.,521/32.,977/788.,977/775.,977/896.,52...",US-94388978-A,"US-3976629-A,US-4101460-A,US-3957698-A,US-3029...",ROHM & HAAS,Emulsion copolymer cation exchange resins,Emulsion copolymer particles with diameters sm...
10,US-4384165-A,US-30179381-A,US,19830517,1981,1981,23164899,large,"H05K5/06,H05K9/00","439/426.,174/355.,439/387.",US-30179381-A,"US-3504095-A,US-3536820-A,US-3904810-A,US-3296...",MOTOROLA INC,Radio frequency shield with force multiplier i...,A high frequency RF gasket is disclosed which ...
11,US-4219679-A,US-95816278-A,US,19800826,1978,1978,25500662,,B01J27/043,"564/422.,568/705.,564/418.",US-95816278-A,"US-3845018-A,US-4051177-A,GB-1431640-A,US-3118...",GULF RESEARCH DEVELOPMENT CO,Selective hydrogenation of nitro groups on nit...,Aromatic nitroacetylene compounds wherein both...
12,US-4219945-A,US-91879078-A,US,19800902,1978,1978,25440973,,"C08J9/04,A43B13/04,A43B13/26,A43B13/20","36/44.,36/29.",US-91879078-A,"US-3410004-A,US-1304915-A,US-2100492-A,US-2739...",BOGERT ROBERT C,Footwear,A shoe embodying a multiple chambered pneumati...
13,US-4439196-A,US-35944782-A,US,19840327,1982,1982,23413825,large,"A61K9/22,A61K9/00","424/473.,604/892.1",US-35944782-A,"US-3982536-A,US-4309996-A,US-3993072-A,US-4093...",MERCK & CO INC,Osmotic drug delivery system,An osmotic drug (or other beneficial substance...


In [11]:
apl_df.to_csv("data/pharma_apl.csv",sep='\t', index=False)